In [ ]:
import pandas as pd
import defillama_utils as dfl
import requests as r
import plotly.express as px

chain = 'Optimism'
tokens = ['USDC','USDT','DAI','SUSD','LUSD','FRAX']
tokens_str = ', '.join(tokens)
print(tokens_str)

In [ ]:
# protos = 'https://api.llama.fi/protocols'
# s = r.Session()
# #get all protocols
# resp = pd.DataFrame( s.get(protos).json() )[['category','slug','chainTvls']]
# display(resp)
# # extract the chain names
# # resp['chainTvls'] = resp['chainTvls'].apply(lambda x: list(x.keys()) )
# # # set a true/false if the array contains the chain we want
# # resp['contains_chain'] = resp['chainTvls'].apply(lambda x: chain_name in x)
# # # filter where we have a match
# # proto_list = resp[['slug']][resp['contains_chain'] == True]
# # # clean up
# # proto_list = proto_list.reset_index(drop=True)
# # # boom


In [ ]:
slugs = dfl.get_protocols_by_chain(chain, exclude_cex = True, exclude_chain = True)
display(slugs)
pds_raw = dfl.get_range(slugs, chain)


In [ ]:
pds = pds_raw.copy()
pds = pds[pds['token'].isin(tokens)]
pds['total_token_tvl'] = pds.groupby(['protocol', 'chain','date'])['usd_value'].\
                                transform('sum').astype(int)

pds['day_rank'] = pds.groupby(['protocol', 'chain'])['date'].\
                            rank(method='dense',ascending=False).astype(int)
pds['usd_value_millions'] = (pds['usd_value'] / 1e6)
display(pds)

In [ ]:
pds_latest = pds[pds['day_rank'] == 1]
pds_latest = pds_latest.sort_values(by = 'total_token_tvl', ascending=False)
display(pds_latest)

In [ ]:
stable_sum = pds_latest[['protocol','usd_value','usd_value_millions']].groupby(['protocol']).sum()
stable_sum = stable_sum.reset_index()
display(stable_sum)

In [ ]:
# stable_sum['usd_value_millions'] = stable_sum['usd_value_millions'].map("${:,.2f}M".format)
chart_pds_latest = pds_latest[pds_latest['total_token_tvl']>1e6]
fig = px.bar(chart_pds_latest, x="protocol", y="usd_value", color="token", title="Protocol TVL by Stablecoin - > $1M in Selected Tokens\
        <br><sup>Note: if the bar is 100% USDT, that means DefiLlama doesn't have splits by token</sup>")
fig.update_layout(barmode='stack', xaxis={'categoryorder': 'total descending'})
fig.show()

fig_pie = px.pie(stable_sum, values='usd_value_millions', names='protocol', title='Share of Stablecoins by Protocol | '+tokens_str)
fig_pie.update_traces(textposition='inside', 
                        text = stable_sum['usd_value_millions'].map("${:,.2f}M".format),
                        textinfo='percent+label+text')
fig_pie.show()